In [1]:
from ultralytics import YOLO
import random, os, tqdm

SRC_IMAGES_PATH = "./unlabeled_images"
DST_LABELS_PATH = "./labels"

if not os.path.exists(DST_LABELS_PATH):
    os.makedirs(DST_LABELS_PATH)

# Initialize a YOLO-World model
model = YOLO('yolov8x-worldv2.pt', verbose=False)  # or choose yolov8m/l-world.pt

## Test on custom Classes

In [9]:
classes = [
    "Rackets"
]
model.set_classes(classes)

In [11]:
IMAGE_PATHS = random.choices([os.path.join(SRC_IMAGES_PATH, im_name) for im_name in os.listdir(SRC_IMAGES_PATH) if os.path.isfile(os.path.join(SRC_IMAGES_PATH, im_name))], k = 1)
predictions = model.predict(IMAGE_PATHS, save = True, stream=False)

print(predictions)

0: 640x640 (no detections), 323.4ms
Speed: 3.0ms preprocess, 323.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: ['Rackets']
obb: None
orig_img: array([[[19, 20, 11],
        [19, 20, 11],
        [19, 20, 11],
        ...,
        [99, 64,  1],
        [99, 64,  1],
        [99, 64,  1]],

       [[19, 20, 11],
        [19, 20, 11],
        [19, 20, 11],
        ...,
        [99, 64,  1],
        [99, 64,  1],
        [99, 64,  1]],

       [[19, 20, 11],
        [19, 20, 11],
        [19, 20, 11],
        ...,
        [99, 64,  1],
        [99, 64,  1],
        [99, 64,  1]],

       ...,

       [[13,  5,  5],
        [13,  5,  5],
        [13,  5,  5],
        ...,
        [13,  3,  3],
        [12,  2,  2],
        [12,  2,  2]],

       [[13,  5,  5],
        [13,  5,  5],
        [

## Train on COCO

In [6]:
classes = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "traffic light", 
    "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", 
    "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", 
    "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", 
    "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", 
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch", 
    "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote", "keyboard", 
    "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", 
    "scissors", "teddy bear", "hair drier", "toothbrush"
]


# Define custom classes
model.set_classes(classes)

In [3]:
batch_size = 20

# IMAGE_PATHS = random.choices([os.path.join(UNLABELED_IMAGES_PATH, im_name) for im_name in os.listdir(UNLABELED_IMAGES_PATH) if os.path.isfile(os.path.join(UNLABELED_IMAGES_PATH, im_name))], k = batch_size)
IMAGE_PATHS = [os.path.join(SRC_IMAGES_PATH, im_name) for im_name in os.listdir(SRC_IMAGES_PATH) if os.path.isfile(os.path.join(SRC_IMAGES_PATH, im_name))]
batches = [IMAGE_PATHS[i:i+batch_size] for i in range(0, len(IMAGE_PATHS), batch_size)]


for batch in tqdm.tqdm(batches, desc="Batch:"):
  # Execute prediction for specified categories on an image
  predictions = model.predict(batch, save = False, stream=True)

  for prediction in predictions:
    image_path = os.path.normpath(prediction.path)
    _, image_name = os.path.split(image_path)
    label_name = os.path.splitext(image_name)[0] + ".txt"
    
    classes = prediction.boxes.cls
    
    with open(os.path.join(DST_LABELS_PATH, label_name), '+w') as file:
      for idx, box in enumerate(prediction.boxes.xywhn):
        cls = int(classes[idx].item())
        file.write(f"{cls} {box[0].item()} {box[1].item()} {box[2].item()} {box[3].item()}\n")

Batch::   0%|          | 0/54 [00:00<?, ?it/s]


0: 640x640 1 person, 1: 640x640 4 bottles, 2: 640x640 (no detections), 3: 640x640 (no detections), 4: 640x640 1 bottle, 5: 640x640 1 bottle, 6: 640x640 1 bottle, 7: 640x640 1 bottle, 8: 640x640 1 bottle, 9: 640x640 5 bottles, 10: 640x640 1 book, 11: 640x640 3 bottles, 1 cup, 12: 640x640 1 book, 13: 640x640 1 bottle, 14: 640x640 1 bottle, 15: 640x640 1 bottle, 1 scissors, 16: 640x640 (no detections), 17: 640x640 1 bottle, 1 cup, 18: 640x640 2 bottles, 1 scissors, 19: 640x640 1 book, 533.3ms
Speed: 1.7ms preprocess, 26.7ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)


Batch::   2%|▏         | 1/54 [00:01<01:36,  1.82s/it]


0: 640x640 1 bottle, 1: 640x640 1 motorcycle, 2: 640x640 1 bottle, 3: 640x640 1 bottle, 1 cup, 4: 640x640 1 book, 2 scissorss, 5: 640x640 1 bottle, 6: 640x640 2 bottles, 7: 640x640 1 parking meter, 8: 640x640 1 bottle, 1 cup, 9: 640x640 1 bottle, 1 cup, 10: 640x640 1 bottle, 11: 640x640 1 bottle, 1 cup, 1 vase, 12: 640x640 2 bottles, 13: 640x640 1 cup, 14: 640x640 1 fire hydrant, 2 bottles, 15: 640x640 1 bottle, 16: 640x640 1 bottle, 17: 640x640 (no detections), 18: 640x640 1 bottle, 19: 640x640 1 handbag, 479.1ms
Speed: 1.5ms preprocess, 24.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Batch::   4%|▎         | 2/54 [00:02<00:59,  1.15s/it]

...

Batch::  96%|█████████▋| 52/54 [00:36<00:01,  1.47it/s]


0: 640x640 1 person, 1 tennis racket, 1: 640x640 2 persons, 1 apple, 1 hair drier, 2: 640x640 1 person, 1 baseball bat, 2 tennis rackets, 3: 640x640 1 person, 1 sports ball, 4: 640x640 2 persons, 1 frisbee, 1 sports ball, 1 baseball glove, 1 tennis racket, 5: 640x640 2 persons, 1 handbag, 6: 640x640 1 scissors, 7: 640x640 1 person, 3 sports balls, 1 baseball glove, 8: 640x640 1 scissors, 9: 640x640 1 scissors, 10: 640x640 2 sports balls, 1 baseball bat, 1 tennis racket, 11: 640x640 1 sports ball, 1 surfboard, 12: 640x640 3 sports balls, 13: 640x640 1 bench, 1 baseball bat, 14: 640x640 1 person, 1 sports ball, 15: 640x640 1 person, 1 tennis racket, 16: 640x640 1 person, 1 frisbee, 1 skateboard, 17: 640x640 1 person, 1 frisbee, 1 tennis racket, 18: 640x640 2 persons, 3 tennis rackets, 19: 640x640 1 person, 1 tennis racket, 1 bottle, 483.8ms
Speed: 1.5ms preprocess, 24.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


Batch::  98%|█████████▊| 53/54 [00:37<00:00,  1.46it/s]


0: 640x640 1 person, 1 tennis racket, 1: 640x640 1 person, 1 baseball bat, 2: 640x640 1 person, 1 tennis racket, 3: 640x640 2 persons, 1 handbag, 1 frisbee, 2 tennis rackets, 1 apple, 4: 640x640 1 person, 5: 640x640 1 person, 6: 640x640 1 person, 2 tennis rackets, 7: 640x640 1 person, 1 tennis racket, 8: 640x640 1 person, 2 tennis rackets, 9: 640x640 1 person, 1 frisbee, 1 sports ball, 2 tennis rackets, 10: 640x640 (no detections), 11: 640x640 1 sports ball, 1 scissors, 12: 640x640 1 scissors, 13: 640x640 2 persons, 2 tennis rackets, 14: 640x640 4 persons, 2 tennis rackets, 15: 640x640 2 persons, 1 tennis racket, 16: 640x640 2 sports balls, 1 tennis racket, 17: 640x640 1 tennis racket, 1 scissors, 18: 640x640 1 bottle, 19: 640x640 1 tennis racket, 483.0ms
Speed: 1.5ms preprocess, 24.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


Batch:: 100%|██████████| 54/54 [00:37<00:00,  1.43it/s]
